In [318]:
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn import preprocessing, svm
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
from sklearn.naive_bayes import GaussianNB
import warnings
warnings.filterwarnings("ignore")

In [291]:
import pandas as pd
pd.set_option("display.max_columns", None)

In [292]:
with open("adult.names") as f:
    print(f.read())

| This data was extracted from the census bureau database found at
| http://www.census.gov/ftp/pub/DES/www/welcome.html
| Donor: Ronny Kohavi and Barry Becker,
|        Data Mining and Visualization
|        Silicon Graphics.
|        e-mail: ronnyk@sgi.com for questions.
| Split into train-test using MLC++ GenCVFiles (2/3, 1/3 random).
| 48842 instances, mix of continuous and discrete    (train=32561, test=16281)
| 45222 if instances with unknown values are removed (train=30162, test=15060)
| Duplicate or conflicting instances : 6
| Class probabilities for adult.all file
| Probability for the label '>50K'  : 23.93% / 24.78% (without unknowns)
| Probability for the label '<=50K' : 76.07% / 75.22% (without unknowns)
|
| Extraction was done by Barry Becker from the 1994 Census database.  A set of
|   reasonably clean records was extracted using the following conditions:
|   ((AAGE>16) && (AGI>100) && (AFNLWGT>1)&& (HRSWK>0))
|
| Prediction task is to determine whether a person makes over

In [293]:
def preprocessing_of_adult_dataset(df):
    for i in range(15):
        df[i]=df[i].astype(str).replace('?',np.NaN)
    df.dropna(subset = [1,6,13],how='any',inplace=True)
    return(df)

In [294]:
def create_y_label(df):
    for i in range(len(df[14])):
        if df[14].iloc[i]=='<=50K' or df[14].iloc[i]=='<=50K.':
            df[14].iloc[i]=1
        elif df[14].iloc[i]=='>50K' or df[14].iloc[i]=='>50K.':
            df[14].iloc[i]=0
    return df[14]

In [295]:
train_df=pd.read_csv('adult.data', sep=',',header=None,skipinitialspace = True)

In [296]:
train_df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [297]:
train_df.shape

(32561, 15)

In [298]:
train_df=preprocessing_of_adult_dataset(train_df)

In [299]:
train_df=train_df.astype({0: 'int64',2:'int64',4:'int64',10: 'int64',11:'int64',12:'int64'}, copy=False)

In [300]:
train_df.isna().sum()

0     0
1     0
2     0
3     0
4     0
5     0
6     0
7     0
8     0
9     0
10    0
11    0
12    0
13    0
14    0
dtype: int64

In [301]:
test_df=pd.read_csv('adult.test', sep=',',header=None,skipinitialspace = True,skiprows=1)

In [302]:
test_df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,25,Private,226802,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States,<=50K.
1,38,Private,89814,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States,<=50K.
2,28,Local-gov,336951,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States,>50K.
3,44,Private,160323,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States,>50K.
4,18,?,103497,Some-college,10,Never-married,?,Own-child,White,Female,0,0,30,United-States,<=50K.


In [303]:
test_df.shape

(16281, 15)

In [304]:
test_df=preprocessing_of_adult_dataset(test_df)

In [305]:
test_df=test_df.astype({0: 'int64',2:'int64',4:'int64',10: 'int64',11:'int64',12:'int64'}, copy=False)

In [306]:
test_df.isna().sum()

0     0
1     0
2     0
3     0
4     0
5     0
6     0
7     0
8     0
9     0
10    0
11    0
12    0
13    0
14    0
dtype: int64

In [307]:
y_test=create_y_label(test_df)
y_train=create_y_label(train_df)

In [308]:
y_test=y_test.astype('int64', copy=False)
y_train=y_train.astype('int64', copy=False)

In [309]:
test_df.drop([14], axis=1,inplace=True)
train_df.drop([14], axis=1,inplace=True)

In [310]:
X_train=pd.get_dummies(train_df)
X_test=pd.get_dummies(test_df)

In [311]:
X_train.drop(['13_Holand-Netherlands'], axis=1,inplace=True)

In [312]:
lgt_R = LogisticRegression(random_state = 101, solver='lbfgs', multi_class='auto')
lgt_R.fit(X_train, y_train)
print('Accuracy:',lgt_R.score(X_test,y_test))

Accuracy: 0.7926958831341302


In [319]:
gnb = GaussianNB()
y_pred = gnb.fit(X_train, y_train).predict(X_test)
print('Accuracy:',gnb.score(X_test,y_test))

Accuracy: 0.7887118193891103
